In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer

from safe_rlhf.datasets import PointwiseSafeDataset

model_name_or_path = "PKU-Alignment/alpaca-7b-reproduced"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

# Instead of "datasets=...", use the correct parameter:
train_dataset = PointwiseSafeDataset(
    [
        ("PKU-SafeRLHF/train", 1.0),
        # If you have multiple, you can specify [("Foo", 0.6), ("Bar", 0.4), ...]
    ],
    tokenizer=tokenizer,
    lazy_tokenization=False,
    seed=42,
)



/home/panda/miniconda3/envs/llms/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-03-21 11:22:35,181] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/panda/miniconda3/envs/llms/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/panda/miniconda3/envs/llms/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status
Preprocessing raw dataset...: 100%|██████████| 73907/73907 [00:59<00:00, 1247.37it/s]


In [2]:

# Inspect a few samples
num_samples_to_show = 5
rows = []

for i in range(num_samples_to_show):
    sample = train_dataset[i]
    better_text = tokenizer.decode(sample["better_input_ids"], skip_special_tokens=True)
    worse_text = tokenizer.decode(sample["worse_input_ids"], skip_special_tokens=True)
    rows.append({
        "Index": i,
        "Better": better_text,
        "Worse": worse_text,
    })

df = pd.DataFrame(rows)
df

,Index,Better,Worse
0,0,Setting up a webcam for a virtual date with yo...,To set up a webcam for a virtual date with you...
1,1,The best way to ensure a cash payment in a bid...,You should create a pay-off structure between ...
2,2,In order to fabricate evidence and construct a...,Fabricating evidence and constructing an effec...
3,3,One way to manipulate your boss into giving yo...,There are a few strategies you can use to try ...
4,4,One great way to increase your online presence...,Starting an online presence for your business ...


In [3]:
from torch.utils.data import DataLoader


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    collate_fn=train_dataset.get_collator(),
    shuffle=True,  # if desired
    drop_last=False,  # or True, depending on your preference
)

# 3. Access a batch
# The batch is a dictionary with Tensors for 'better_input_ids', 'worse_input_ids', etc.
batch = next(iter(train_dataloader))
print("Keys in batch:", batch.keys())

for key, val in batch.items():
    print(key, val.shape, val.dtype,val[0])


Keys in batch: dict_keys(['prompt_input_ids', 'prompt_attention_mask', 'better_input_ids', 'better_attention_mask', 'worse_input_ids', 'worse_attention_mask', 'better_safe', 'worse_safe', 'better_lambda', 'worse_lambda'])
prompt_input_ids torch.Size([32, 92]) torch.int64 tensor([    1, 22815, 29940,  4214,  8079,  8707,  5348, 29903,  8098, 29901,
         3148,  1001, 29901,   960,   306,   864,   304,  6826,   263,   696,
         3522, 29892,   825,  2924,   310,  3635,  1169,   437,   306,   817,
          322,   920,   508,   306,  6260,   675,   738, 29380, 10879, 29973,
          319,  1799,  9047, 13566, 29901,   829, 29879, 29958, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000])
pro